## Coursera - IBM Data Science Capstone Project ##

### Brian Lee ###
BrianTrewLee@gmail.com<br />
LinkedIn: https://www.linkedin.com/in/brian-lee-64a2022b/



A description of the problem and a discussion of the background. (15 marks)<br />
A description of the data and how it will be used to solve the problem. (15 marks)

## 1. Introduction ##

If I were to open a Starbucks, where should I locate it? Clearly, the location is one of the most important business decisions for this venture. Is there a science to locating a StarBucks? Can we create a machine learning model to predict where a StarBucks is likely to be located? I'd like to find out.

If a reliable model can be made,it could be used in the process of opening a store. It could be used as a final sanity check, or at the beginning stage, to select a promising location. If a trustworthy model predicts with a high degree of confidence that a Starbucks should be located in an area, but there is not one there yet, perhaps that is an opportunity.

Any person interested in opening a Starbucks should be interested in these results. This includes myself, other potential franchisees, and the Starbucks corporation itself, which operates many of it's own stores. The Starbucks competition might also be interested as they could possibly gain competitive insigths. I also believe others might be interested in this procedure, as it might be applied to predicting the location of other entities, based on the same sort of data.

The main purpose of this project is to prove the concept of predicting Starbuck's locations in general. I suspect it may work better or worse depending on the locations chosen for training and prediction, the radius size, and the specific features used in the model. I may vary those factors in order to prove the concept, which could then be applied carefully to a particular geographic area of interest at a later date.

### 2. Data ###

Foursquare (https://foursquare.com) is a company that crowdsources location data, tying latitude-longitude coordinates (and other things) with public venues, including many businesses, such as Starbucks locations. The features for the Starbucks location classifier will come from venue data from the Fourquare API. I will use venue category names (such as 'bar', 'Chinese restaurant', 'coffee shop') as features to classify an area as either an area with a Starbucks, or an area without one. I will use the venue name (e.g. 'Starbucks') to determine whether or not the venue is a Starbucks. An area containing a Starbucks contains at least one venue named Starbucks within the radius supplied to the Foursquare API 'explore' endpoint.

An area will be a circular area with a given radius. I have not determined that radius yet, and I might use more than one radius in this project. Within that radius, the Foursquare API will return venues. The larger the radius, the more venues that could be anticipated, and the higher likelihood of a Starbucks, but a larger radius is also less useful for business use. I have done some exploratory work with a radius of 300m.

The areas to be chosen should contain a mix of areas with Starbuck's and those without. I have done some exploratory work in a particular area of San Francisco. I located Starbucks with Google Maps and got latitude and longitude points making a box around that area. I could choose the train and test points at random within an area, or I could generate a grid covering an entire area. Given the constraints my Foursquare Developer account places on my searches, and knowing that binary classifiers work better with a good number of examples of both classes, I may want to generate areas that I know will be heavy with Starbucks locations. To that end, I discovered a Kaggle dataset (https://www.kaggle.com/starbucks/store-locations) containing the latitude and longitude coordinates of 25,600 worldwide Starbucks locations. I may or may not use this resource.

### 3. Methodology ###

I will generate a corpus of geographic points, and using a reasonable radius (perhaps 300m) a corpus of geographic areas. Each geographic area will have a feature set based on json data returned by the Foursquare API's 'explore' endpoint. I plan to use the category names of the returned venues as features for that area. I will use the venue names to determine whether oir not a Starbucks is in that area, and generate the labels from that. 

Once I have labelled data, I will use sklearn.model_selection.train_test_split to generate tarining and testing sets. Then I  can use any of a host of sklearn classifiers to generate predictive models. I have done some exploratory work with a random forest.

In [1]:
# import libraries

import urllib.request # open and read URLs

import json # handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # handle requests
import pandas as pd # process data as dataframes with Pandas
import numpy as np # handle data in a vectorized manner with NumPy

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't installed the GeoPy geocoding library yet
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't installed the Folium library yet
import folium # map rendering library

# Matplolib plotting library and associated modules
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # for K-Means clustering with Scikit-Learn

print("Libraries imported!")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
# Define a function to gather recommended venues, with specifically the name and category, using the explore API

def getNearbyVenues(latitudes, longitudes, radius=300, limit=100):
#def getNearbyVenues(latitudes, longitudes, radius=300, limit=100):
#def getNearbyVenues(latitudes, longitudes, radius=750, limit=100):
#def getNearbyVenues(latitudes, longitudes, radius=2000, limit=100):
    
    count = 0
    venues_list = []
    for lat, lng in zip(latitudes, longitudes):
        #create a unique namefor this pointthat is easier todeal with than a combination of lat and lon
        point_name = "point"+str(count)
        print(point_name,':',lat,'-',lng)
         

            
        # create the API request URL
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        #print("resultscols=",results.columns)
        

        
        # return only relevant information for each nearby venue
        venues_list.append([(
            point_name,
            lat, 
            lng, 
            v["venue"]["name"], 
            v["venue"]["location"]["lat"], 
            v["venue"]["location"]["lng"],  
            v["venue"]["location"]["distance"], 
            v["venue"]["categories"][0]["name"]) for v in results])
        count = count + 1
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Point Name",
                  "Neighborhood Latitude", 
                  "Neighborhood Longitude", 
                  "Venue", 
                  "Venue Latitude", 
                  "Venue Longitude", 
                  "Venue Distance",           
                  "Venue Category"]
    
    return(nearby_venues)

In [4]:
sf_lat=37.7749
sf_lon=-122.4194
ch_lat=41.8781
ch_lon=-87.6298
la_lat=34.0522
la_lon=-118.2437
starbucks_m_lat=37.744644
starbucks_m_lon=-122.452731
import random

#37.780253, -122.509117
#37.804203, -122.459264 - upper left of starbucks intensive space
#37.767547, -122.409342 - lower right ofstarbucksintensive space
llat=37.767547
ulat=37.804203
llon=-122.459264
ulon=-122.409342
random_lat0 = random.uniform(37.767547,37.804203)
random_lon0 = random.uniform(-122.459264, -122.409342)
random_lat1 = random.uniform(llat,ulat)
random_lon1 = random.uniform(llon, ulon)
random_lat2 = random.uniform(llat,ulat)
random_lon2 = random.uniform(llon, ulon)
random_lat3 = random.uniform(llat,ulat)
random_lon3 = random.uniform(llon, ulon)
random_lat4 = random.uniform(llat,ulat)
random_lon4 = random.uniform(llon, ulon)
random_lat5 = random.uniform(llat,ulat)
random_lon5 = random.uniform(llon, ulon)
random_lat6 = random.uniform(llat,ulat)
random_lon6 = random.uniform(llon, ulon)
random_lat7 = random.uniform(llat,ulat)
random_lon7 = random.uniform(llon, ulon)
random_lat8 = random.uniform(llat,ulat)
random_lon8 = random.uniform(llon, ulon)
random_lat9 = random.uniform(llat,ulat)
random_lon9 = random.uniform(llon, ulon)

data = pd.DataFrame(np.array([[sf_lat, sf_lon], [ch_lat, ch_lon], [la_lat, la_lon], [starbucks_m_lat,starbucks_m_lon], [random_lat0,random_lon0], [random_lat1,random_lon1], [random_lat2,random_lon2], [random_lat3,random_lon3], [random_lat4,random_lon4], [random_lat5,random_lon5], [random_lat6,random_lon6], [random_lat7,random_lon7], [random_lat8,random_lon8], [random_lat9,random_lon9] ]), columns=['Latitude', 'Longitude'])

data



,Latitude,Longitude
0,37.774900,-122.419400
1,41.878100,-87.629800
2,34.052200,-118.243700
3,37.744644,-122.452731
4,37.777190,-122.413290
5,37.784248,-122.443936
6,37.798106,-122.420917
7,37.769420,-122.421316
8,37.790511,-122.453908
9,37.787545,-122.456537


In [5]:
random_latlons = []
for j in range(0,300):
    random_lat = random.uniform(llat,ulat)
    random_lon = random.uniform(llon,ulon)
    random_latlon=(random_lat,random_lon)
    random_latlons.append(random_latlon)
    

In [6]:
random_latlons = pd.DataFrame(random_latlons)
random_lats=random_latlons[0]
random_lons=random_latlons[1]
random_lats.shape



(300,)

In [7]:
random_latlons[0].shape

(300,)

In [8]:
random_latlons[1].shape

(300,)

In [9]:


data["Latitude"] = random_latlons[0]
data["Longitude"] = random_latlons[1]

data["Latitude"].head()
data["Latitude"].shape


(14,)

In [10]:
data["Longitude"].head()
data.shape

(14, 2)

In [11]:

#venues = getNearbyVenues(latitudes = data["Latitude"],
#                               longitudes = data["Longitude"]
#                               )
venues = getNearbyVenues(latitudes = random_lats,
                               longitudes = random_lons
                               )


point0 : 37.795583403873174 - -122.41182435055846


KeyError: 'groups'

In [ ]:
venues.tail()

In [ ]:
venues['Venue'].unique()
venues['Venue'].value_counts()

In [ ]:
venues[350:400]


# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['point_name'] = venues['Point Name'] 
onehot['point_lat'] = venues['Neighborhood Latitude'] 
onehot['point_lon'] = venues['Neighborhood Longitude'] 
onehot['isStarbucks'] = (venues['Venue'] == "Starbucks")
# move neighborhood column to the first column
fixed_columns = [onehot.columns[-3]] + [onehot.columns[-2]] + [onehot.columns[-1]] + list(onehot.columns[:-3])
onehot = onehot[fixed_columns]

onehot.isStarbucks = onehot.isStarbucks.astype(int)

onehot.head()
onehot['isStarbucks'].value_counts()

In [ ]:
venues.head()

In [ ]:
grouped = onehot.groupby('point_name').mean().reset_index()
grouped

In [ ]:
num_top_venues = 50

for point in grouped['point_name']:
    print("----"+point+"----")
    temp = grouped[grouped['point_name'] == point].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
grouped.head()

In [ ]:
y = grouped['isStarbucks'] > 0
y = pd.DataFrame(y)
y = y.isStarbucks.astype(int)
print(y.value_counts())
X = grouped
print(X.shape)
X .drop(columns=['point_name', 'point_lat', 'point_lon', 'isStarbucks'], inplace=True)
print(X.columns)
print(X.shape)

In [ ]:
from sklearn.model_selection import train_test_split
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
import sklearn as sk  
from sklearn.ensemble import RandomForestClassifier

#RF = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)  
RF = RandomForestClassifier(n_estimators=3, max_depth=30, random_state=0)  
RF.fit(X, y)  
RF.predict(X)  
RF.score(X,y)

In [ ]:
estimator = RF.estimators_[2]

In [ ]:
estimator

In [ ]:
from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = X.columns,
                class_names = 'has_starbucks',
                rounded = True, proportion = False, 
                precision = 2, filled = True)

In [ ]:
# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')